In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("/content/100_Unique_QA_Dataset.csv")
data.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [8]:
df=data

In [5]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [6]:
vocab = {'<UNK>':0}

In [7]:
def built_vocab(row):
  tokenised_question=tokenize(row['question'])
  tokenised_answer=tokenize(row['answer'])

  merged_tokens=tokenised_question+tokenised_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token]=len(vocab)

In [9]:
df.apply(built_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [11]:
len(vocab)

324

In [12]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [13]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [16]:
import torch
from torch.utils.data import  Dataset, DataLoader


In [24]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [25]:
dataset=QADataset(df,vocab)

In [29]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [26]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [27]:
for question, answer in dataloader:
  print(question, answer)

tensor([[ 42, 107,   2, 108,  19, 109]]) tensor([[110]])
tensor([[  1,   2,   3, 141, 117,  83,   3, 277, 278]]) tensor([[121]])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([[246]])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([[131]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([[106]])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([[156]])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([[74]])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([[36]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[78, 79, 80, 81, 82, 83, 84]]) tensor([[85]])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([[16]])
tensor([[ 42, 216, 118, 217, 218,  19,  14, 219,  43]]) tensor([[220]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[  1,   2,   3,   4,   5, 279]]) tensor([[280]])
tensor([[ 1,  2,  3, 17, 18, 19, 20, 21, 22]]) tensor([[23]])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor(

In [30]:
import torch.nn as nn
import torch.optim as optim

In [36]:
class SimpleRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,50)
    self.rnn=nn.RNN(50,64,batch_first=True)
    self.fc=nn.Linear(64,vocab_size)

  def forward(self, question):
    embebbed_question=self.embedding(question)
    hidde, final=self.rnn(embebbed_question)
    output = self.fc(final.squeeze(0))

    return output

In [37]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [38]:
learning_rate=0.01
epochs=20

In [39]:
model=SimpleRNN(len(vocab))

In [40]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [41]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 533.622496
Epoch: 2, Loss: 334.058657
Epoch: 3, Loss: 153.654074
Epoch: 4, Loss: 61.050875
Epoch: 5, Loss: 35.364933
Epoch: 6, Loss: 25.710049
Epoch: 7, Loss: 18.925917
Epoch: 8, Loss: 24.670096
Epoch: 9, Loss: 12.707864
Epoch: 10, Loss: 18.385605
Epoch: 11, Loss: 4.975962
Epoch: 12, Loss: 1.256873
Epoch: 13, Loss: 0.659682
Epoch: 14, Loss: 0.529446
Epoch: 15, Loss: 0.453187
Epoch: 16, Loss: 0.400061
Epoch: 17, Loss: 0.358936
Epoch: 18, Loss: 0.324650
Epoch: 19, Loss: 0.295451
Epoch: 20, Loss: 0.271285


In [42]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [44]:
predict(model,"which is the largest planer in solar system?")

jupiter
